In [1]:
!pip install transformers datasets accelerate evaluate

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.8 MB/s eta 0:00:00


In [2]:
!pip install sentence_transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 5.4 MB/s eta 0:00:00


In [3]:
from sentence_transformers import InputExample
import tqdm.auto as tqdm

def convert_example(dataset):
  pairs = []
  for example in tqdm.tqdm(dataset):
    pairs.append(InputExample(
    texts=[example['sentence1'], example['sentence2']],
    label=example['labels']['binary-label']))
  return pairs

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [5]:
#전처리
from datasets import load_dataset
sts_train = load_dataset('klue', 'sts', split='train')
sts_eval = load_dataset('klue', 'sts', split='validation')

train_examples = convert_example(sts_train)
eval_examples = convert_example(sts_eval)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.52M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/68.8k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/11668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/519 [00:00<?, ? examples/s]

  0%|          | 0/11668 [00:00<?, ?it/s]

  0%|          | 0/519 [00:00<?, ?it/s]

In [6]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=2)

In [7]:
from sentence_transformers import SentenceTransformer
from sentence_transformers .models import Transformer, Pooling

word_emb_model = Transformer('klue/bert-base', max_seq_length=256)
pooling_model = Pooling(word_emb_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_emb_model, pooling_model])

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from sentence_transformers.losses import ContrastiveLoss
train_loss = ContrastiveLoss(model=model)

In [9]:
model.fit([(train_dataloader, train_loss)], show_progress_bar=True)

Step,Training Loss
500,0.015500
1000,0.013700
1500,0.010000
2000,0.008200
2500,0.008500
3000,0.007300
3500,0.007300
4000,0.007700
4500,0.006400
5000,0.006700


In [10]:
from sentence_transformers.evaluation import BinaryClassificationEvaluator
eval = BinaryClassificationEvaluator.from_input_examples(eval_examples)
eval.compute_metrices(model)

{'cosine': {'accuracy': 0.8342967244701349,
  'accuracy_threshold': 0.7741590738296509,
  'f1': 0.828,
  'f1_threshold': 0.7453417778015137,
  'precision': 0.7392857142857143,
  'recall': 0.9409090909090909,
  'ap': 0.8566533633890985},
 'dot': {'accuracy': 0.8053949903660886,
  'accuracy_threshold': 333.7709655761719,
  'f1': 0.7956204379562044,
  'f1_threshold': 289.5284423828125,
  'precision': 0.6646341463414634,
  'recall': 0.990909090909091,
  'ap': 0.8232955271218485},
 'manhattan': {'accuracy': 0.838150289017341,
  'accuracy_threshold': 305.13787841796875,
  'f1': 0.8225469728601252,
  'f1_threshold': 308.4602355957031,
  'precision': 0.7606177606177607,
  'recall': 0.8954545454545455,
  'ap': 0.8527101219802009},
 'euclidean': {'accuracy': 0.838150289017341,
  'accuracy_threshold': 13.580739974975586,
  'f1': 0.8208333333333334,
  'f1_threshold': 14.047199249267578,
  'precision': 0.7576923076923077,
  'recall': 0.8954545454545455,
  'ap': 0.8517732692469986}}

In [11]:
pretrained_model = SentenceTransformer('jhgan/ko-sroberta-nli')
eval.compute_metrices(pretrained_model)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.49k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

{'cosine': {'accuracy': 0.8323699421965318,
  'accuracy_threshold': 0.791296124458313,
  'f1': 0.8160676532769556,
  'f1_threshold': 0.791296124458313,
  'precision': 0.7628458498023716,
  'recall': 0.8772727272727273,
  'ap': 0.843465397046822},
 'dot': {'accuracy': 0.8188824662813102,
  'accuracy_threshold': 102.32653045654297,
  'f1': 0.8041237113402062,
  'f1_threshold': 96.86182403564453,
  'precision': 0.7358490566037735,
  'recall': 0.8863636363636364,
  'ap': 0.8447704663582504},
 'manhattan': {'accuracy': 0.8323699421965318,
  'accuracy_threshold': 160.75051879882812,
  'f1': 0.8176100628930818,
  'f1_threshold': 161.57769775390625,
  'precision': 0.7587548638132295,
  'recall': 0.8863636363636364,
  'ap': 0.8386150988954901},
 'euclidean': {'accuracy': 0.8265895953757225,
  'accuracy_threshold': 7.275073528289795,
  'f1': 0.810924369747899,
  'f1_threshold': 7.299893379211426,
  'precision': 0.75390625,
  'recall': 0.8772727272727273,
  'ap': 0.8380055205019963}}